In [2]:
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.quantum_info import Pauli
from qiskit.circuit import Parameter
from qiskit.quantum_info import Statevector, Operator
from qiskit.visualization import array_to_latex
from qiskit import Aer, execute
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import PauliEvolutionGate
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt

from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.settings import QiskitNatureSettings

from qiskit_algorithms import TrotterQRTE
from qiskit_algorithms import TimeEvolutionProblem


QiskitNatureSettings.use_pauli_sum_op = False





import numpy as np

In [3]:
# Setup

M = 2  #Number of Localizations
S = 2  #Number of Spins
L = 2  #Number of Sites
N = M*S*L #total number of operators (Two positions times two spin polarizations)

zero = Operator(np.zeros((2**N,2**N)))


In [23]:
# Basis states

TuTu = Statevector.from_int(17, 2**N)
TuTd = Statevector.from_int(18, 2**N)
TdTu = Statevector.from_int(33, 2**N)
TdTd = Statevector.from_int(34, 2**N)

TuBu = Statevector.from_int(20, 2**N)
TuBd = Statevector.from_int(24, 2**N)
TdBu = Statevector.from_int(36, 2**N)
TdBd = Statevector.from_int(40, 2**N)

BuTu = Statevector.from_int(65, 2**N)
BuTd = Statevector.from_int(66, 2**N)
BdTu = Statevector.from_int(127, 2**N)   
BdTd = Statevector.from_int(128, 2**N)

BuBu = Statevector.from_int(68, 2**N)
BuBd = Statevector.from_int(72, 2**N)
BdBu = Statevector.from_int(132, 2**N)
BdBd = Statevector.from_int(136, 2**N)


TuTd.draw('latex')


<IPython.core.display.Latex object>

In [26]:
# Define tunneling operators

# tunnel_op_list = [
#     ({"+_0 -_5": 1.0}, 8),
#     ({"+_0 -_6": 1.0}, 8),
#     ({"+_0 -_7": 1.0}, 8),
#     ({"+_1 -_4": 1.0}, 8),
#     ({"+_1 -_6": 1.0}, 8),
#     ({"+_1 -_7": 1.0}, 8),
#     ({"+_2 -_4": 1.0}, 8),
#     ({"+_2 -_5": 1.0}, 8),
#     ({"+_2 -_7": 1.0}, 8),
#     ({"+_3 -_4": 1.0}, 8),
#     ({"+_3 -_5": 1.0}, 8),
#     ({"+_3 -_6": 1.0}, 8),
#     ({"+_4 -_1": 1.0}, 8),
#     ({"+_4 -_2": 1.0}, 8),
#     ({"+_4 -_3": 1.0}, 8),
#     ({"+_5 -_0": 1.0}, 8),
#     ({"+_5 -_2": 1.0}, 8),
#     ({"+_5 -_3": 1.0}, 8),
#     ({"+_6 -_0": 1.0}, 8),
#     ({"+_6 -_1": 1.0}, 8),
#     ({"+_6 -_3": 1.0}, 8),
#     ({"+_7 -_0": 1.0}, 8),
#     ({"+_7 -_1": 1.0}, 8),
#     ({"+_7 -_2": 1.0}, 8),
# ]


second_order_tunnel = [
    ({"+_0 -_4 +_4 -_0": 1.0}, 8),
    ({"+_0 -_4 +_5 -_1": 1.0}, 8),
    ({"+_0 -_4 +_6 -_2": 1.0}, 8),
    ({"+_0 -_4 +_7 -_3": 1.0}, 8),
    ({"+_1 -_5 +_4 -_0": 1.0}, 8),
    ({"+_1 -_5 +_5 -_1": 1.0}, 8),
    ({"+_1 -_5 +_6 -_2": 1.0}, 8),
    ({"+_1 -_5 +_7 -_3": 1.0}, 8),
    ({"+_2 -_6 +_4 -_0": 1.0}, 8),
    ({"+_2 -_6 +_5 -_1": 1.0}, 8),
    ({"+_2 -_6 +_6 -_2": 1.0}, 8),
    ({"+_2 -_6 +_7 -_3": 1.0}, 8),
    ({"+_3 -_7 +_4 -_0": 1.0}, 8),
    ({"+_3 -_7 +_5 -_1": 1.0}, 8),
    ({"+_3 -_7 +_6 -_2": 1.0}, 8),
    ({"+_3 -_7 +_7 -_3": 1.0}, 8),
    ({"+_4 -_0 +_0 -_4": 1.0}, 8),
    ({"+_4 -_0 +_1 -_5": 1.0}, 8),
    ({"+_4 -_0 +_2 -_6": 1.0}, 8),
    ({"+_4 -_0 +_3 -_7": 1.0}, 8),
    ({"+_5 -_1 +_0 -_4": 1.0}, 8),
    ({"+_5 -_1 +_1 -_5": 1.0}, 8),
    ({"+_5 -_1 +_2 -_6": 1.0}, 8),
    ({"+_5 -_1 +_3 -_7": 1.0}, 8),
    ({"+_6 -_2 +_0 -_4": 1.0}, 8),
    ({"+_6 -_2 +_1 -_5": 1.0}, 8),
    ({"+_6 -_2 +_2 -_6": 1.0}, 8),
    ({"+_6 -_2 +_3 -_7": 1.0}, 8),
    ({"+_7 -_3 +_0 -_4": 1.0}, 8),
    ({"+_7 -_3 +_1 -_5": 1.0}, 8),
    ({"+_7 -_3 +_2 -_6": 1.0}, 8),
    ({"+_7 -_3 +_3 -_7": 1.0}, 8),
]

first_order_tunnel = [
    ({"+_0 -_4": 1.0}, 8),
    ({"+_1 -_5": 1.0}, 8),
    ({"+_2 -_6": 1.0}, 8),
    ({"+_3 -_7": 1.0}, 8),
    ({"+_4 -_0": 1.0}, 8),
    ({"+_5 -_1": 1.0}, 8),
    ({"+_6 -_2": 1.0}, 8),
    ({"+_7 -_3": 1.0}, 8)
]



In [38]:
tunnel_hamiltonian_op = {}
for i in range(0, len(first_order_tunnel)):
    tunnel_hamiltonian_op.update(first_order_tunnel[i][0])

tunnel_hamiltonian_op

{'+_0 -_4': 1.0,
 '+_1 -_5': 1.0,
 '+_2 -_6': 1.0,
 '+_3 -_7': 1.0,
 '+_4 -_0': 1.0,
 '+_5 -_1': 1.0,
 '+_6 -_2': 1.0,
 '+_7 -_3': 1.0}

In [40]:
tunnel_hamiltonian = JordanWignerMapper().map(FermionicOp(tunnel_hamiltonian_op))
tunnel_hamiltonian

SparsePauliOp(['IIIYZZZY', 'IIIXZZZX', 'IIYZZZYI', 'IIXZZZXI', 'IYZZZYII', 'IXZZZXII', 'YZZZYIII', 'XZZZXIII'],
              coeffs=[0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j])

In [44]:
# Time evolution

initial_state = TuTu

num_steps = 10
time_list = np.linspace(0, 1, num_steps)
delta_t = time_list[1] - time_list[0]

# Store states in each step
state_list = []
state_list.append(initial_state)

for i in tqdm(range(num_steps-1)):
    problem = TimeEvolutionProblem(tunnel_hamiltonian, initial_state=state_list[-1], time=delta_t)
    trotter = TrotterQRTE()
    result = trotter.evolve(problem)
    state_list.append(Statevector(result.evolved_state))

# Gather state occupations as probabilities
prob_dict = {"00010001": [], "00010010": [], "00010100": [], "00011000": [], "00100001": [], "00100010": [], "00100100": [], "00101000": [], 
             "01000001": [], "01000010": [], "01000100": [], "01001000": [], "10000001": [], "10000010": [], "10000100": [], "10001000": []}




  0%|          | 0/9 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [41]:
TdTu = TuTd.evolve(JordanWignerMapper().map(FermionicOp(second_order_tunnel[1][0], num_spin_orbitals=N)) + JordanWignerMapper().map(FermionicOp(second_order_tunnel[21][0], num_spin_orbitals=N)))

TdTu.draw('latex')

<IPython.core.display.Latex object>